In [ ]:
%pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 8.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### I have Scrapped the Knowledge base and cleaned it before creating vector embeddings (The data used here is a small subset of the original data)

In [ ]:
import pandas as pd
import numpy as np
import json
from sentence_transformers import SentenceTransformer

# Load the CSV data
data = pd.read_csv('/content/drive/MyDrive/RAG CHATBOT/cs_du_scraped_data_clean - DEMO.csv')  # Replace with your CSV file path

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
embeddings = []
texts = []

for index, row in data.iterrows():
    text = row['cleaned_content']  # Replace with the name of your text column
    texts.append(text)
    embedding = model.encode(text)
    embeddings.append(embedding)

# Convert to numpy array for easier handling
embeddings = np.array(embeddings)

# Save embeddings
np.save('csdu_embeddings.npy', embeddings)

# Save corresponding texts
with open('csdu_texts.json', 'w') as f:
    json.dump(texts, f)

print(f"Created and saved {len(embeddings)} embeddings")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Created and saved 6 embeddings


### Created Vector Database

In [ ]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 61.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import faiss
import json

# Load the embeddings and texts
embeddings = np.load('csdu_embeddings.npy')

# Ensure embeddings are in float32 format for FAISS
embeddings = embeddings.astype('float32')

with open('csdu_texts.json', 'r') as f:
    texts = json.load(f)

# Create a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add the vectors to the index
index.add(embeddings)

# Save the index
faiss.write_index(index, 'csdu_faiss_index.index')  # Added '.index' extension for clarity

print(f"Created and saved FAISS index with {index.ntotal} vectors")

Created and saved FAISS index with 6 vectors


### RAG Implementation Starts

In [ ]:
!pip install requests python-dotenv

In [ ]:
import requests
from sentence_transformers import SentenceTransformer
from typing import List
from dotenv import load_dotenv

In [ ]:
 import getpass

API KEY - "Generate your own"

In [ ]:
HUGGINGFACE_API_KEY = getpass.getpass('Enter your Hugging Face API key: ')


Enter your Hugging Face API key: ··········


In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 23.2 MB/s eta 0:00:00


### Google API Key used : " Generate your own API KEY "

In [ ]:
google_api_key = getpass.getpass('Enter your Google API key: ')

Enter your Google API key: ··········


In [ ]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
import google.generativeai as genai

class RAGChatbot:
    def __init__(self, index_path, texts_path, google_api_key,
                 embedding_model_name="all-MiniLM-L6-v2",
                 generation_model_name="gemini-pro"):
        # Load FAISS index
        self.index = faiss.read_index(index_path)

        # Load texts
        with open(texts_path, 'r') as f:
            self.texts = json.load(f)

        # Initialize sentence transformer for encoding queries
        self.embedding_model = SentenceTransformer(embedding_model_name)

        # Configure Google Gemini API
        genai.configure(api_key=google_api_key)

        # Initialize Gemini model
        self.generation_model = genai.GenerativeModel(generation_model_name)

    def get_query_embedding(self, query):
        embedding = self.embedding_model.encode([query])[0].astype('float32')
        return embedding

    def get_relevant_context(self, query, k=3, max_tokens=800):
        query_vector = self.get_query_embedding(query)
        distances, indices = self.index.search(np.array([query_vector]), k)
        relevant_texts = [self.texts[i] for i in indices[0]]
        context = " ".join(relevant_texts)

        char_limit = max_tokens * 4
        if len(context) > char_limit:
            context = context[:char_limit]

        return context

    def generate_response(self, context, question):
        prompt = f"""You are an intelligent assistant designed to help students, faculty, and visitors interact with the Department of Computer Science at the University of Delhi (cs.du.ac.in). Your responses should be based on the following context and guidelines:

Context: {context}

Guidelines:
1. Provide detailed, accurate, and clear responses based on the given context and your knowledge about the department.
2. Always be polite, helpful, and ensure your answers are well-structured.
3. If you don't have enough information to answer a query, politely suggest contacting the department directly.
4. Tailor your language to be appropriate for academic settings while remaining accessible to a diverse audience.
5. When discussing courses, research, or facilities, be as specific as possible based on the available information.
6. If asked about events or deadlines, remind the user to verify the most current information on the official website.

Question: {question}

Answer:
"""

        # Generate content using the generation model
        response = self.generation_model.generate_content(prompt)
        answer = response.text

        # Footer to be appended to the generated answer
        footer = """
#### Contact Information:
- **Address**: Department of Computer Science, Faculty of Mathematical Sciences, 1st Floor, New Academic Block, University of Delhi, Delhi 110007, India.
- **Phone**: +91-11-2766-7591, 2766-7059
- **Email**: office@cs.du.ac.in
- **Website**: [Department of Computer Science, University of Delhi](https://cs.du.ac.in)
---
**Note**: The content on this page is subject to the copyright of the University of Delhi, 2022.
**Website Development Team**: [Site Map](https://cs.du.ac.in/sitemap)
For more information, visit the [University of Delhi](https://www.du.ac.in) website.
"""

        # Return the combined response
        return answer + footer

    def chat(self, query):
        context = self.get_relevant_context(query)
        answer = self.generate_response(context, query)
        return answer


In [ ]:
chatbot = RAGChatbot(
    index_path='csdu_faiss_index.index',
    texts_path='csdu_texts.json',
    google_api_key=google_api_key,  # Using Google API key for Gemini
    embedding_model_name="sentence-transformers/all-MiniLM-L6-v2",
    generation_model_name="gemini-pro"  # Matching your original configuration for the Google Gemini model
)

In [ ]:
def ask_chatbot(question):
    try:
        answer = chatbot.chat(question)
        print(f"Question: {question}\n")
        print(f"Answer: {answer}\n")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
ask_chatbot("How is the MCA course at the University of Delhi?")

Question: How is the MCA course at the University of Delhi?

Answer: The Master of Computer Applications (MCA) program at the University of Delhi is designed to provide a comprehensive and in-depth understanding of computer science. The program emphasizes the fundamentals of computing, equipping students with the skills and knowledge necessary to succeed in the IT industry.

The MCA program at Delhi University is highly regarded for its rigorous curriculum, experienced faculty, and state-of-the-art facilities. The program is structured to provide a strong foundation in computer science principles, while also offering a range of elective courses that allow students to specialize in areas of their interest.

The program's curriculum is regularly updated to reflect the latest advancements in the field of computer science. The faculty is highly qualified and experienced, and they are dedicated to providing students with the best possible learning experience. The department also has excelle

In [ ]:
ask_chatbot("How is the PhD course at the University of Delhi?")

Question: How is the PhD course at the University of Delhi?

Answer: The Ph.D. program in Computer Science at the University of Delhi is designed to prepare students for a career in research and development. The program offers a rigorous curriculum that covers a wide range of topics in computer science, including:

* Algorithms and Complexity
* Artificial Intelligence
* Computer Architecture
* Databases
* Graphics and Visualization
* Networking
* Operating Systems
* Programming Languages
* Software Engineering
* Theory of Computation

Students in the program work closely with experienced faculty members who are actively engaged in research. They have the opportunity to participate in research projects, attend conferences, and publish their work in leading academic journals.

The program typically takes 5-7 years to complete. Students must complete coursework, pass a qualifying examination, and write and defend a dissertation.

The Ph.D. program in Computer Science at the University of 

In [ ]:
ask_chatbot("What are the current research areas available in the PhD program?")

Question: What are the current research areas available in the PhD program?

Answer: The Department of Computer Science at the University of Delhi offers a wide range of research areas in its Ph.D. program. For the most up-to-date information, please refer to the "List of Ph.D. Supervisors / Research Areas" available at the following link:

https://cs.du.ac.in/uploads/phd-cs/List-of-Supervisor-Co-SupervisorJuly2023.pdf
#### Contact Information:
- **Address**: Department of Computer Science, Faculty of Mathematical Sciences, 1st Floor, New Academic Block, University of Delhi, Delhi 110007, India.
- **Phone**: +91-11-2766-7591, 2766-7059
- **Email**: office@cs.du.ac.in
- **Website**: [Department of Computer Science, University of Delhi](https://cs.du.ac.in)
---
**Note**: The content on this page is subject to the copyright of the University of Delhi, 2022.
**Website Development Team**: [Site Map](https://cs.du.ac.in/sitemap)
For more information, visit the [University of Delhi](https://w

In [ ]:
  ask_chatbot("What are the areas of publication at Department of Computer Science University of Delhi? I am interested towards algorithms ")

Question: What are the areas of publication at Department of Computer Science University of Delhi? I am interested towards algorithms 

Answer: The Department of Computer Science at the University of Delhi has published research in various areas, including Algorithms. Some notable publications in this field include:

* **Algorithms for Network Optimization**
* **Approximation Algorithms for NP-complete Problems**
* **Online Algorithms for Scheduling and Resource Allocation**
* **Graph Algorithms and Their Applications**

If you are specifically interested in Algorithms, I recommend exploring the research of Professor Neelima Gupta, who is a faculty member in this area. You can find more information on the department's website: https://cs.du.ac.in

Remember to check the website for the most up-to-date information on research publications and faculty expertise.
#### Contact Information:
- **Address**: Department of Computer Science, Faculty of Mathematical Sciences, 1st Floor, New Academ

In [ ]:
ask_chatbot(("Tell me eerything you know about Department of Computer Science , University of Delhi?"))

Question: Tell me eerything you know about Department of Computer Science , University of Delhi?

Answer: The Department of Computer Science at the University of Delhi, established in 1981, is renowned for its high-quality education in computer science. It offers a range of programs, including:

**Programs Offered:**

- Master of Computer Applications (MCA) - 3 years
- MSc Computer Science - 2 years
- PhD in Computer Science
- Undergraduate Programs:
    - BSc Hons Computer Science
    - BSc Physical and Mathematical Sciences
    - BA Program Elective Courses in Computer Applications

**Key Facilities and Initiatives:**

- Modern infrastructure for teaching, learning, and research
- Extensive research areas, including:
    - Artificial Intelligence and Machine Learning
    - Data Science and Analytics
    - Blockchain Technology
    - Cloud Computing
    - Natural Language Processing
    - Computer Vision
- Dedicated research faculty, including several PhD supervisors
- Various student

In [ ]:
ask_chatbot("Professor Neelima Gupta?")

Question: Professor Neelima Gupta?

Answer: Sure! Professor Neelima Gupta is a faculty member of the Department of Computer Science at the University of Delhi. She is an experienced and knowledgeable professor specializing in Algorithms and Theoretical Computer Science. I recommend visiting her profile page on the department's website for more information about her research interests and publications. You can also contact her via email for specific inquiries or to schedule a meeting.
#### Contact Information:
- **Address**: Department of Computer Science, Faculty of Mathematical Sciences, 1st Floor, New Academic Block, University of Delhi, Delhi 110007, India.
- **Phone**: +91-11-2766-7591, 2766-7059
- **Email**: office@cs.du.ac.in
- **Website**: [Department of Computer Science, University of Delhi](https://cs.du.ac.in)
---
**Note**: The content on this page is subject to the copyright of the University of Delhi, 2022.
**Website Development Team**: [Site Map](https://cs.du.ac.in/sitema

In [ ]:
ask_chatbot('Neelima Gupta research areas')

Question: Neelima Gupta research areas

Answer: Professor Neelima Gupta is a faculty member in the Department of Computer Science at the University of Delhi. Her current research areas include:
- Algorithms
#### Contact Information:
- **Address**: Department of Computer Science, Faculty of Mathematical Sciences, 1st Floor, New Academic Block, University of Delhi, Delhi 110007, India.
- **Phone**: +91-11-2766-7591, 2766-7059
- **Email**: office@cs.du.ac.in
- **Website**: [Department of Computer Science, University of Delhi](https://cs.du.ac.in)
---
**Note**: The content on this page is subject to the copyright of the University of Delhi, 2022.
**Website Development Team**: [Site Map](https://cs.du.ac.in/sitemap)
For more information, visit the [University of Delhi](https://www.du.ac.in) website.




In [ ]:
ask_chatbot('tell me something interesting about DUCS?')

Question: tell me something interesting about DUCS?

Answer: The Department of Computer Science (DUCS) at the University of Delhi has been at the forefront of academic excellence and research in computer science for over four decades. Here are a few notable points:

**Faculty Expertise:** 
DUCS boasts a team of highly accomplished faculty members who are actively involved in cutting-edge research and teaching. Their expertise spans a wide range of areas, including artificial intelligence, computer networks, data mining, information security, and theoretical computer science.

**Research Recognition:** 
DUCS faculty have a strong publication record in top-tier journals and conferences. The department has consistently received research grants from various funding agencies, including the University Grants Commission (UGC), Department of Science and Technology (DST), and Science and Engineering Research Board (SERB).

**Innovative Curriculum:** 
DUCS offers a comprehensive curriculum that 

In [ ]:
# class RAGChatbot:
#     def __init__(self, index_path, texts_path, huggingface_api_key, google_api_key=None,
#                  embedding_model_name="all-MiniLM-L6-v2",
#                  generation_model_name="google/gemini"):
#         # Load FAISS index
#         self.index = faiss.read_index(index_path)

#         # Load texts
#         with open(texts_path, 'r') as f:
#             self.texts = json.load(f)

#         # Initialize sentence transformer for encoding queries
#         self.embedding_model = SentenceTransformer(embedding_model_name)

#         # Hugging Face API details
#         self.api_key = huggingface_api_key
#         self.headers = {"Authorization": f"Bearer {self.api_key}"}

#         # Google API details (if needed)
#         self.google_api_key = google_api_key

#         # Define model endpoints
#         self.embedding_endpoint = f"https://api-inference.huggingface.co/models/{embedding_model_name}"
#         self.generation_endpoint = f"https://api-inference.huggingface.co/models/{generation_model_name}"

#     def get_query_embedding(self, query):
#         embedding = self.embedding_model.encode([query])[0].astype('float32')
#         return embedding

#     def get_relevant_context(self, query, k=3, max_tokens=800):
#         query_vector = self.get_query_embedding(query)
#         distances, indices = self.index.search(np.array([query_vector]), k)
#         relevant_texts = [self.texts[i] for i in indices[0]]
#         context = " ".join(relevant_texts)

#         char_limit = max_tokens * 4
#         if len(context) > char_limit:
#             context = context[:char_limit]

#         return context

#     def generate_response(self, context, question):
#         prompt = (
#             f"You are an intelligent assistant that provides detailed and accurate answers based on the provided context.\n\n"
#             f"Context: {context}\n\n"
#             f"Question: {question}\n\n"
#             f"Answer:"
#         )

#         payload = {
#             "inputs": prompt,
#             "parameters": {
#                 "max_new_tokens": 150,
#                 "temperature": 0.7,
#                 "top_p": 0.9,
#                 "do_sample": True,
#                 "repetition_penalty": 1.03,
#                 "stop": ["\n"]
#             }
#         }

#         response = requests.post(self.generation_endpoint, headers=self.headers, json=payload)

#         if response.status_code == 200:
#             generated = response.json()
#             answer = generated.get('generated_text', '').split("Answer:")[-1].strip()
#             return answer
#         else:
#             raise Exception(f"Request failed with status code {response.status_code}: {response.text}")

#     def chat(self, query):
#         context = self.get_relevant_context(query)
#         answer = self.generate_response(context, query)
#         return answer

In [ ]:
# class RAGChatbot:
#     def __init__(self, index_path, texts_path, huggingface_api_key, embedding_model_name="all-MiniLM-L6-v2",
#                  generation_model_name="EleutherAI/gpt-neo-2.7B"):
#         # Load FAISS index
#         self.index = faiss.read_index(index_path)

#         # Load texts
#         with open(texts_path, 'r') as f:
#             self.texts = json.load(f)

#         # Initialize sentence transformer for encoding queries
#         self.embedding_model = SentenceTransformer(embedding_model_name)

#         # Hugging Face API details
#         self.api_key = huggingface_api_key
#         self.headers = {"Authorization": f"Bearer {self.api_key}"}

#         # Define model endpoints
#         self.embedding_endpoint = f"https://api-inference.huggingface.co/models/{embedding_model_name}"
#         self.generation_endpoint = f"https://api-inference.huggingface.co/models/{generation_model_name}"

#     def get_query_embedding(self, query):
#         # Using local embedding model for speed and to stay within free tier
#         embedding = self.embedding_model.encode([query])[0].astype('float32')
#         return embedding

#     def get_relevant_context(self, query, k=3, max_tokens=800):
#         # Encode the query
#         query_vector = self.get_query_embedding(query)

#         # Search the FAISS index
#         distances, indices = self.index.search(np.array([query_vector]), k)

#         # Retrieve the corresponding texts
#         relevant_texts = [self.texts[i] for i in indices[0]]

#         # Join the relevant texts
#         context = " ".join(relevant_texts)

#         # Optionally, truncate context based on token limit (assuming 4 chars per token)
#         char_limit = max_tokens * 4
#         if len(context) > char_limit:
#             context = context[:char_limit]

#         return context

#     def generate_response(self, context, question):
#         # Prepare the payload for the language model
#         prompt = (
#             f"You are an intelligent assistant that provides detailed and accurate answers based on the provided context.\n\n"
#             f"Context: {context}\n\n"
#             f"Question: {question}\n\n"
#             f"Answer:"
#         )

#         payload = {
#             "inputs": prompt,
#             "parameters": {
#                 "max_new_tokens": 150,
#                 "temperature": 0.7,
#                 "top_p": 0.9,
#                 "do_sample": True,
#                 "repetition_penalty": 1.03,
#                 "stop": ["\n"]
#             }
#         }

#         response = requests.post(self.generation_endpoint, headers=self.headers, json=payload)

#         if response.status_code == 200:
#             generated = response.json()
#             if isinstance(generated, list):
#                 answer = generated[0]['generated_text'].split("Answer:")[-1].strip()
#             else:
#                 answer = generated.get('generated_text', '').split("Answer:")[-1].strip()
#             return answer
#         else:
#             raise Exception(f"Request failed with status code {response.status_code}: {response.text}")

#     def chat(self, query):
#         context = self.get_relevant_context(query)
#         answer = self.generate_response(context, query)
#         return answer


In [ ]:
# import os
# import numpy as np
# import faiss
# import json
# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import time

# class RAGChatbot:
#     def __init__(self, index_path, texts_path, model_name="gpt2"):
#         # Load FAISS index
#         self.index = faiss.read_index('csdu_faiss_index.index')

#         # Load texts
#         with open(texts_path, 'r') as f:
#             self.texts = json.load(f)

#         # Initialize sentence transformer for encoding queries
#         self.sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

#         # Initialize language model and tokenizer
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model = AutoModelForCausalLM.from_pretrained(model_name)

#     def get_relevant_context(self, query, k=3):
#         start_time = time.time()

#         # Encode the query
#         query_vector = self.sentence_transformer.encode([query])[0].astype('float32')
#         print(f"Query encoding took: {time.time() - start_time} seconds")

#         # Search the FAISS index
#         start_time = time.time()
#         _, indices = self.index.search(np.array([query_vector]), k)
#         print(f"FAISS search took: {time.time() - start_time} seconds")

#         # Retrieve the corresponding texts
#         relevant_texts = [self.texts[i] for i in indices[0]]

#         return " ".join(relevant_texts)

#     def generate_response(self, query, context):
#         # Prepare input for the language model
#         input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
#         input_ids = self.tokenizer.encode(input_text, return_tensors="pt")
#         print(f"Input token length: {len(input_ids[0])}")

#         # Generate response
#         start_time = time.time()
#         output = self.model.generate(input_ids, max_length=300, num_return_sequences=1, no_repeat_ngram_size=2)
#         print(f"Model generation took: {time.time() - start_time} seconds")

#         response = self.tokenizer.decode(output[0], skip_special_tokens=True)

#         # Extract the generated answer (everything after "Answer:")
#         answer = response.split("Answer:")[-1].strip()

#         return answer

#     def chat(self, query):
#         context = self.get_relevant_context(query)
#         response = self.generate_response(query, context)
#         return response

# # Usage
# chatbot = RAGChatbot('csdu_faiss_index.index', 'csdu_texts.json')

# # Example conversation loop
# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ['quit', 'exit', 'bye']:
#         break
#     response = chatbot.chat(user_input)
#     print(f"Chatbot: {response}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# import os
# import numpy as np
# import faiss
# import json
# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModelForCausalLM

# class RAGChatbot:
#     def __init__(self, index_path, texts_path, model_name="gpt2"):
#         # Load FAISS index
#         self.index = faiss.read_index(index_path)

#         # Load texts
#         with open(texts_path, 'r') as f:
#             self.texts = json.load(f)

#         # Initialize sentence transformer for encoding queries
#         self.sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

#         # Initialize language model and tokenizer
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model = AutoModelForCausalLM.from_pretrained(model_name)

#     def get_relevant_context(self, query, k=3, max_tokens=1500):
#         # Encode the query
#         query_vector = self.sentence_transformer.encode([query])[0].astype('float32')

#         # Search the FAISS index
#         _, indices = self.index.search(np.array([query_vector]), k)

#         # Retrieve the corresponding texts
#         relevant_texts = [self.texts[i] for i in indices[0]]

#         # Join the relevant texts
#         context = " ".join(relevant_texts)

#         # Truncate the context if it exceeds the token limit
#         context_tokens = self.tokenizer.encode(context)

#         if len(context_tokens) > max_tokens:
#             context_tokens = context_tokens[:max_tokens]  # Truncate to max_tokens
#             context = self.tokenizer.decode(context_tokens, skip_special_tokens=True)

#         return context

#     def generate_response(self, query, context, max_new_tokens=150):
#         # Prepare input for the language model
#         input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
#         input_ids = self.tokenizer.encode(input_text, return_tensors="pt")

#         # Generate response
#         output = self.model.generate(input_ids, max_new_tokens=max_new_tokens, num_return_sequences=1, no_repeat_ngram_size=2)

#         response = self.tokenizer.decode(output[0], skip_special_tokens=True)

#         # Extract the generated answer (everything after "Answer:")
#         answer = response.split("Answer:")[-1].strip()

#         return answer

#     def chat(self, query):
#         context = self.get_relevant_context(query)
#         response = self.generate_response(query, context)
#         return response

# # Usage
# chatbot = RAGChatbot('csdu_faiss_index.index', 'csdu_texts.json')

# # Example conversation loop
# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ['quit', 'exit', 'bye']:
#         break
#     response = chatbot.chat(user_input)
#     print(f"Chatbot: {response}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1552 > 1024). Running this sequence through the model will result in indexing errors
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: It is a course that is offered in English and is taught in a language other than English. It has a focus on computer programming and the development of a computer system. Students will learn to program and develop their own systems.
...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: .
,
 (1) The department is a research institution, with a focus on computer sciences. It is the largest research university in Delhi. Its research is focused on the development of new technologies and applications for human and machine intelligence. In addition, it is also the only university to have a Computer Sciences Research Institute (CRSI) and a Research and Development Center (RDC). The CSRSIs are responsible for developing and implementing new research and development tools for computer systems. They are the main source of funding for research in this field. CSRIs also provide funding to other research institutes and universities. For example, in 2012, a CSRI was awarded to a university for its research on artificial intelligence and artificial neural networks


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: MCS 3-3 years
.
 (1) Introduction to Computer Engineering
: Introduction
, Introduction, Computer Architecture, Design, Programming, Data Structures
- Introduction: Design and Implementation of Applications
 and Applications, Application Architecture and Data Structure
 - Introduction and Design of Data
...
"The MSC Computer Sciences program is a unique opportunity for students of all levels of engineering to learn the fundamental concepts of programming, data structures, algorithms, programming languages, machine learning and machine intelligence. It is an opportunity to gain a deep understanding and appreciation of these concepts and to build a foundation for a career in engineering." - Professor, Department, Faculty of Engineering, Delhi University, India
The Department is committed to providing students the best possible


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: Yes, you can take MSC at any university in Delhi.
.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: We are currently looking for a faculty member to join our faculty at UCD. We have a number of candidates who are interested in joining our department, but are not yet ready to apply. Please contact us at:
.
,
-
:
